In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st
import pprint
# Import API key

from api_keys import g_key
gmaps.configure(api_key=g_key)

# Data File path (CSV)
#data = "./output_data/cities.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path)
print(Crime_df)


       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [3]:
#Clean the Zipcode Data
Crime_df = Crime_df.dropna(subset=['ZIP'])
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [4]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]
print(Zipcodes_df)

          ZIP
0       78205
1       78201
2       78218
3       78201
4       78207
...       ...
219219  78220
219220  78220
219221  78222
219222  78257
219223  78228

[217469 rows x 1 columns]


In [5]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
Num_of_Crimes_2 = Num_of_Crimes.astype(float)
Num_of_Crimes_3 = pd.DataFrame(Num_of_Crimes_2)
print(type(Num_of_Crimes_3))
# print(Num_of_Crimes_2)
print(Num_of_Crimes_3)

<class 'pandas.core.frame.DataFrame'>
       INCIDENT_NUM
ZIP                
0               4.0
1               3.0
11111           6.0
50701           2.0
6278            6.0
...             ...
78291           1.0
78404           1.0
78648           1.0
79502           3.0
81507           1.0

[113 rows x 1 columns]


In [6]:
Zipcodes_df_2 = Zipcodes_df["ZIP"].unique()
print(Zipcodes_df_2)
print(type(Zipcodes_df_2))

['78205' '78201' '78218' '78207' '78225' '78210' '78230' '78212' '78211'
 '78228' '78242' '78221' '78238' '78213' '78202' '78245' '78240' '78233'
 '78219' '78224' '78251' '78216' '78237' '78217' '78209' '78227' '78214'
 '78254' '78223' '78252' '78220' '78266' '78247' '78232' '78249' '78204'
 '78256' '78250' '78229' '78235' '78203' '78215' '78258' '78253' '78226'
 '78244' '78239' '78023' '78154' '78208' '78222' '78109' '78259' '78257'
 '78130' '78255' '78248' '11111' '78121' '78260' '78231' '78207-3170'
 '78236' '78241' '78155' '78070' '79502' '78233-6610' '78073' '78147' '0'
 '78112' '78234' '1' '78223-2099' '78016' '78000' '50701' '6278' '81507'
 '78264' '78148' '78263' '78056' '78006' '78' '78205-2687' '78261' '78160'
 '78291' '78404' '78132' '78201-3847' '78108' '78026' '78009' '78238-4205'
 '78288' '78216-6902' '78054' '78163' '78200' '78015' '78002' '78025'
 '70205' '78227-3794' '78648' '78284' '78014' '78101' '74134' '78069']
<class 'numpy.ndarray'>


In [7]:
Zipcodes_df_2 = pd.DataFrame(Zipcodes_df_2, columns=['Zip'])

Zipcodes_df_2["Lat"] = ""
Zipcodes_df_2["Lon"] = ""

# Target zipcode
# Zipcodes_df_2["Zip"] = Zipcodes_df_2["ZIP"].astype(str)

print(Zipcodes_df_2.dtypes)
print(Zipcodes_df_2)

Zip    object
Lat    object
Lon    object
dtype: object
       Zip Lat Lon
0    78205        
1    78201        
2    78218        
3    78207        
4    78225        
..     ...  ..  ..
108  78284        
109  78014        
110  78101        
111  74134        
112  78069        

[113 rows x 3 columns]


In [8]:
# params = {"query": Zipcode,"key": g_key}




for index,row in Zipcodes_df_2.iterrows():

    # get zipcode info from df
    Zipcode = row['Zip']
    
    # add keyword to params dict
#     params['query'] = Zipcode
#     params['key'] = g_key
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    # Build the endpoint URL
    target_url = ("https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(Zipcode)+"&key="+g_key)
    response = requests.get(target_url).json()
#     pprint.pprint(response)
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"The lat is: ['results'][0]['geometry']['location']['lat']")
        print(f"The lng is: ['results'][0]['geometry']['location']['lng']")
        
        Zipcodes_df_2.loc[index, 'Lat'] = response['results'][0]["geometry"]["location"]["lat"]
        Zipcodes_df_2.loc[index, 'Lon'] = response['results'][0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Zipcodes_df_2

Retrieving Results for Index 0: 78205.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 1: 78201.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 2: 78218.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 3: 78207.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 4: 78225.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 5: 78210.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
-----

The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 50: 78222.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 51: 78109.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 52: 78259.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 53: 78257.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 54: 78130.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Ind

The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 98: 78216-6902.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 99: 78054.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 100: 78163.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
------------
Retrieving Results for Index 101: 78200.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng']
Missing field/result... skipping.
------------
Retrieving Results for Index 102: 78015.
The lat is: ['results'][0]['geometry']['location']['lat']
The lng is: ['results'][0]['geometry']['location']['lng

,Zip,Lat,Lon
0,78205,29.4227,-98.487
1,78201,29.4636,-98.5227
2,78218,29.4896,-98.3855
3,78207,29.4175,-98.5227
4,78225,29.3856,-98.5254
...,...,...,...
108,78284,29.5331,-98.4033
109,78014,28.4613,-99.1013
110,78101,29.3378,-98.2322
111,74134,36.113,-95.7827


In [14]:
latitude = pd.to_numeric(Zipcodes_df_2["Lat"])
longitude = pd.to_numeric(Zipcodes_df_2["Lon"])
print(len(latitude))
print(len(longitude))

locations = (latitude, longitude)
wei = Num_of_Crimes_3
print(len(Crime))

113
113
113


In [13]:
fig= gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Crime, dissipating=False,
                                point_radius=5)

fig.add_layer(heat_layer)

fig

ValueError: weights must be of the same length as locations or None